In [1]:
# toxic-comments.csv
# https://mitu.co.in/dataset

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Embedding, GRU, LSTM
from keras.utils import plot_model

In [3]:
df = pd.read_csv('toxic-comments.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df.shape

(159571, 8)

In [5]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

### Define target variables ( adapt based on your needs )

In [6]:
 toxicities = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [7]:
def clean_text(text):
    text = text.lower() #lower
    text = re.sub(r'[^a-zA-Z0-9]+', ' ', text) #remove non-alphanumeric characters
    return text

In [8]:
df['comment_text'] = df['comment_text'].apply(clean_text)

### Define input and output variables 

In [9]:
# features and target preparation
comments = df['comment_text'].tolist()
target = df[toxicities].values

In [10]:
target.shape

(159571, 6)

### Prepare the data

In [11]:
# Tokenization and padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(comments)
sequences = tokenizer.texts_to_sequences(comments)
padded_sequences = pad_sequences(sequences, maxlen=200)

### Cross-validation

In [12]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, target, test_size=0.2, random_state=0)

In [13]:
X_train.shape

(127656, 200)

In [14]:
X_test.shape

(31915, 200)

In [15]:
# Model definition
model = Sequential()
model.add(Embedding(5000, 120, input_length=200))
model.add(GRU(64))
model.add(Dense(6, activation='sigmoid'))
#multiclass

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 120)          600000    
                                                                 
 gru (GRU)                   (None, 64)                35712     
                                                                 
 dense (Dense)               (None, 6)                 390       
                                                                 
Total params: 636102 (2.43 MB)
Trainable params: 636102 (2.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### compile the model

In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train the model

In [19]:
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/3


3990/3990 [==============================] - 438s 109ms/step - loss: 0.0626 - accuracy: 0.9821 - val_loss: 0.0507 - val_accuracy: 0.9936
Epoch 2/3
3990/3990 [==============================] - 433s 109ms/step - loss: 0.0457 - accuracy: 0.9860 - val_loss: 0.0486 - val_accuracy: 0.9919
Epoch 3/3
3990/3990 [==============================] - 433s 108ms/step - loss: 0.0407 - accuracy: 0.9545 - val_loss: 0.0510 - val_accuracy: 0.9898


### Predict on new data

In [20]:
#new_comment = 'This is an awful and offensive comment!'
new_comment = 'For someone writing a blog about how not to be a douche online, you do a pretty good job of being a total douche!'

In [21]:
new_sequence = tokenizer.texts_to_sequences([clean_text(new_comment)])
padded_new_sequence = pad_sequences(new_sequence, maxlen=200)
prediction = model.predict(padded_new_sequence)[0]

1/1 [==============================] - 1s 652ms/step


In [22]:
prediction

array([2.1883567e-01, 6.8572949e-04, 1.7348785e-02, 9.1329690e-05,
       1.4559539e-01, 8.8937255e-04], dtype=float32)

In [23]:
for toxicity, prob in zip(toxicities, prediction):
    print(f"{toxicity}: {prob:.2f}")

toxic: 0.22
severe_toxic: 0.00
obscene: 0.02
threat: 0.00
insult: 0.15
identity_hate: 0.00
